In [118]:
import pandas as pd

In [124]:
from pymongo import MongoClient
import time
import api_keys
import requests
import gdax
import json

api = api_keys.api_keys()

client = MongoClient(api.mongo)
# specify the database and collection`
db = client.gdax.gdaxws
print('connected to mongo collection')

# and store in local variables
key = api.api_key
b64secret = api.secret_key
passphrase = api.passphrase

class GAC(gdax.AuthenticatedClient):
    def buy(self, **kwargs):
        kwargs["side"] = "buy"
        r = requests.post(self.url + '/orders',
                          data=json.dumps(kwargs),
                          auth=self.auth,
                          timeout=30)

        print(r)
        return r.json()

    def sell(self, **kwargs):
        kwargs["side"] = "sell"
        r = requests.post(self.url + '/orders',
                          data=json.dumps(kwargs),
                          auth=self.auth,
                          timeout=30)

        print('SOLD !!')
        print(r)
        db.orders.insert_one(r.json())
        return r.json()

    def getorders(self):
        """{'id': 'e22c9172-0276-47f7-b774-2559784c26aa', 'price': '999.85000000', 
            'size': '0.01000000', 'product_id': 'BTC-EUR', 'side': 'buy', 
            'type': 'limit', 'time_in_force': 'GTC', 'post_only': True,
            'created_at': '2018-08-17T14:57:18.551778Z', 'fill_fees': '0.0000000000000000',
            'filled_size': '0.00000000', 'executed_value': '0.0000000000000000', 
            'status': 'open', 'settled': False}"""
    
        self.openorders = client.get_orders()[0]
        self.df_openorders = pd.DataFrame(self.openorders)
        
    def getbalances(self):
        """{'id': '459d001f-0391-4e97-89e7-ae474275e2c9', 'currency': 'BTC',
            'balance': '0.0530287336346057', 'available': '0.0530287336346057',
            'hold': '0.0000000000000000', 'profile_id': '5100622b-3ed2-49e4-9810-c28fb96d30b3'} """

        self.balances = client.get_accounts()
        self.df_balances = pd.DataFrame(self.balances)

connected to mongo collection


In [125]:
client = GAC(key=key,b64secret=b64secret,
                 passphrase=passphrase,
                 api_url="https://api.pro.coinbase.com")
    
client.getbalances()

EUR balance at 164.2057972936818500
BTC balance at 0.0530287336346057


In [140]:
client.df_balances

,available,balance,currency,hold,id,profile_id
0,0.0530287336346057,0.0530287336346057,BTC,0.0000000000000000,459d001f-0391-4e97-89e7-ae474275e2c9,5100622b-3ed2-49e4-9810-c28fb96d30b3
1,0,0.0000000000000000,LTC,0.0000000000000000,ad1b7960-0dcb-440e-b60a-73ab56ab1035,5100622b-3ed2-49e4-9810-c28fb96d30b3
2,111.20729729368185,164.2057972936818500,EUR,52.9985000000000000,0a0fd378-71d0-4d5c-b7bb-0b48b1ff839c,5100622b-3ed2-49e4-9810-c28fb96d30b3
3,0,0.0000000000000000,ETH,0.0000000000000000,175a9d3f-e634-4575-bee7-96594bbaed3a,5100622b-3ed2-49e4-9810-c28fb96d30b3
4,0,0.0000000000000000,ETC,0.0000000000000000,a355123a-d206-4992-af06-20dce476611e,5100622b-3ed2-49e4-9810-c28fb96d30b3
5,0,0.0000000000000000,BCH,0.0000000000000000,5ea49af8-18ad-45d7-bf2f-e1f3b4bc7358,5100622b-3ed2-49e4-9810-c28fb96d30b3


In [167]:
balances = client.df_balances.query('currency in ("BTC","EUR")').T
balances.rename(columns=balances.loc['currency'], inplace=True)

In [170]:
balances.rename(columns=balances.loc['currency'], inplace=True)

In [154]:
bal

In [171]:
balances

,BTC,EUR
available,0.0530287336346057,111.20729729368185
balance,0.0530287336346057,164.2057972936818500
currency,BTC,EUR
hold,0.0000000000000000,52.9985000000000000
id,459d001f-0391-4e97-89e7-ae474275e2c9,0a0fd378-71d0-4d5c-b7bb-0b48b1ff839c
profile_id,5100622b-3ed2-49e4-9810-c28fb96d30b3,5100622b-3ed2-49e4-9810-c28fb96d30b3


In [648]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import time
import api_keys
import requests
import gdax
import random as random
import json
import pandas as pd

api = api_keys.api_keys()

client = MongoClient(api.mongo)
# specify the database and collection`
db = client.gdax.gdaxws
print("connected to mongo collection")

# and store in local variables
key = api.api_key
b64secret = api.secret_key
passphrase = api.passphrase

class clearingmaster():
    def __init__(self, maxopenbuyorders = 4 ,**kwargs):
        self.maxopenbuyorders = maxopenbuyorders
        self.heartbeat_rate = 10  # seconds before we auto cancel limit order
        self.marketBTCUSD = client.get_product_ticker("BTC-USD")
        self.marketBTCEUR = client.get_product_ticker("BTC-EUR")
        self.exchangerate = float(self.marketBTCEUR["price"]) / float(self.marketBTCUSD["price"])

    def getorders(self):
        """{'id': 'e22c9172-0276-47f7-b774-2559784c26aa', 'price': '999.85000000', 
            'size': '0.01000000', 'product_id': 'BTC-EUR', 'side': 'buy', 
            'type': 'limit', 'time_in_force': 'GTC', 'post_only': True,
            'created_at': '2018-08-17T14:57:18.551778Z', 'fill_fees': '0.0000000000000000',
            'filled_size': '0.00000000', 'executed_value': '0.0000000000000000', 
            'status': 'open', 'settled': False}"""
    
        openorders = client.get_orders()[0]
        try:
            df_openorders = pd.DataFrame(openorders).query('product_id == "BTC-EUR"')
            df_openorders['created_at'] = pd.to_datetime(df_openorders['created_at'])
            self.df_openorders = df_openorders
            return True
        except:
            print('DAXY no orders on books')
            self.df_openorders = pd.DataFrame(None)
            return False
        
    def getbalances(self):
        """{'id': '459d001f-0391-4e97-89e7-ae474275e2c9', 'currency': 'BTC',
            'balance': '0.0530287336346057', 'available': '0.0530287336346057',
            'hold': '0.0000000000000000', 'profile_id': '5100622b-3ed2-49e4-9810-c28fb96d30b3'} """

        balances = client.get_accounts()
        df_balances = pd.DataFrame(balances).query('currency in ("BTC","EUR")').T
        df_balances.rename(columns=df_balances.loc['currency'], inplace=True)
        
        l = ['available','balance','hold']        
        self.df_balances = df_balances.loc[l].astype('float')
        #print('DAXY DEBUG', self.df_balances)

    def getclearance(self, kwargs_dict):
        try:
            orderprice = kwargs_dict['price'] * kwargs_dict['size']
            print('DAXY CM order received total price: {}'.format(orderprice))
            self.getbalances()
            self.getorders()
        except KeyError:
            print('DAXY CM KeyError calculating orderprice')
            print('DAXY CM KeyError rejecting order request')
            return False

        if self.df_balances['EUR']['available'] <= orderprice:
            print('DAXY CM insufficient funds')
            return False
        elif len(self.df_openorders) >= self.maxopenbuyorders:
            print('DAXY CM maximum limit orders reached')
            return False
        else:
            return True

    def heartbeat(self, **kwargs):
        has_orders = self.getorders()
        if has_orders is True:
            cutoffdate = pd.to_datetime(time.time()-self.heartbeat_rate, unit='s')
            to_terminate = self.df_openorders[self.df_openorders["created_at"]<cutoffdate]['id']
            for idtoterminate in to_terminate:
                response_cancel = client.cancel_order(idtoterminate) 
                db.delete_one({'MONGOKEY':'BUY_ORDER',
                               'trade_id': idtoterminate})
                print('DAXY CANCELLED old order : {}'.format(response_cancel))
        else:
            print('DAXY no orders on books')

class GAC(gdax.AuthenticatedClient):
    def __clearingmaster__(self):
        self.cm_ = clearingmaster()
        self.timeout = 15

    def buy(self, **kwargs):
        """client.buy(size="0.005000000",
                product_id="BTC-EUR",
                side="buy",
                stp="dc",
                type="limit")
                
            from CB PRO website
                {"id": "d0c5340b-6d6c-49d9-b567-48c4bfca13d2",
                "price": "0.10000000",
                "size": "0.01000000",
                "product_id": "BTC-USD",
                "side": "buy",
                "stp": "dc",
                "type": "limit",
                "time_in_force": "GTC",
                "post_only": false,
                "created_at": "2016-12-08T20:02:28.53864Z",
                "fill_fees": "0.0000000000000000",
                "filled_size": "0.00000000",
                "executed_value": "0.0000000000000000",
                "status": "pending",
                "settled": false}"""

        kwargs["side"] = "buy"
        kwargs["type"] = "limit"
        trade_request = self.cm_.getclearance(kwargs_dict=kwargs)
        
        if trade_request == True:
            r = requests.post(self.url + '/orders',
                            data=json.dumps(kwargs),
                            auth=self.auth,
                            timeout=30)

            rjson = r.json()
            rjson.update({'MONGOKEY':'BUY_ORDER','timestamp':time.time(),'trade_id':rjson['id']})
            rjson.pop('id', None)
            db.insert_one(rjson)
            print('DAXY BOUGHT inserted in mongo')
            return rjson
        else:
            print('DAXY CM no permission')

    def sell(self, **kwargs):
        """client.sell(size="0.005000000",
                product_id="BTC-EUR",
                side="sell",
                stp="dc",
                type="limit")"""

        kwargs["side"] = "sell"
        r = requests.post(self.url + '/orders',
                          data=json.dumps(kwargs),
                          auth=self.auth,
                          timeout=30)

        print('SOLD !!')
        print(r)
        r.update({'MONGOKEY':'SELL_ORDER','timestamp':time.time()})
        db.insert_one(r.json())
        print('DAXY SOLD inserted in mongo')
        return r.json()

if __name__ == "__main__":
    client = GAC(key=key,b64secret=b64secret,
                 passphrase=passphrase,
                 api_url="https://api.pro.coinbase.com")
    
    client.__clearingmaster__() # set clearing master
    client.cm_.getorders()
    p_change = {}
    counter=0

connected to mongo collection
DAXY no orders on books


In [ ]:
client = GAC(key=key,b64secret=b64secret,
                 passphrase=passphrase,
                 api_url="https://api.pro.coinbase.com")
    
client.__clearingmaster__() # set clearing master
client.cm_.getorders()

In [541]:
openorders = client.get_orders()[0]

In [543]:
dd = pd.DataFrame(openorders)

In [658]:
client.cm_.heartbeat()

DAXY CANCELLED old order : ['70c4a7c5-e4c9-4684-ab4a-f6155b7a2351']


In [652]:
client.buy(size=0.005000000,price=4091,
                product_id="BTC-EUR",
                side="buy",
                type="limit")

DAXY CM order received total price: 20.455000000000002
DAXY no orders on books
DAXY BOUGHT inserted in mongo


{'MONGOKEY': 'BUY_ORDER',
 '_id': ObjectId('5b7744612d105a6f698332e9'),
 'created_at': '2018-08-17T21:55:45.536191Z',
 'executed_value': '0.0000000000000000',
 'fill_fees': '0.0000000000000000',
 'filled_size': '0.00000000',
 'post_only': False,
 'price': '4091.00000000',
 'product_id': 'BTC-EUR',
 'settled': False,
 'side': 'buy',
 'size': '0.00500000',
 'status': 'pending',
 'stp': 'dc',
 'time_in_force': 'GTC',
 'timestamp': 1534542945.618804,
 'trade_id': '70c4a7c5-e4c9-4684-ab4a-f6155b7a2351',
 'type': 'limit'}

In [660]:
client.cm_.getorders()
client.cm_.df_openorders

DAXY no orders on books


""


In [676]:
df = pd.DataFrame(
        list(db.find({'MONGOKEY' :'BUY_ORDER'})\
                        .sort([('timestamp', 1)]))) 
df

,MONGOKEY,_id,created_at,executed_value,fill_fees,filled_size,post_only,price,product_id,settled,side,size,status,stp,strategy,time_in_force,timestamp,trade_id,type
0,BUY_ORDER,5b776aaa1d8d9b0001935509,2018-08-18T00:39:06.168525Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5729.09000000,BTC-EUR,False,buy,0.00300000,pending,dc,ISS,GTC,1.534553e+09,c20dd74d-87c5-4a08-840c-aae8137628a6,limit
1,BUY_ORDER,5b776aaa3f8dc70001432777,2018-08-18T00:39:06.179289Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5729.09000000,BTC-EUR,False,buy,0.00300000,pending,dc,ISS,GTC,1.534553e+09,3b93b502-2532-438b-940e-3f1ba982ac52,limit
2,BUY_ORDER,5b776aab2d105a029aabbb48,2018-08-18T00:39:07.675635Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5737.99000000,BTC-EUR,False,buy,0.00100000,pending,dc,ISS,GTC,1.534553e+09,83f1943e-4628-4878-92f3-974648bc534f,limit
3,BUY_ORDER,5b776adb1d8d9b000193550a,2018-08-18T00:39:55.883861Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5730.20000000,BTC-EUR,False,buy,0.00300000,pending,dc,ISS,GTC,1.534553e+09,42e61b4c-ba51-4fa9-a62c-2beb5a8c3e9a,limit
4,BUY_ORDER,5b776adb3f8dc70001432778,2018-08-18T00:39:55.93461Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5730.20000000,BTC-EUR,False,buy,0.00300000,pending,dc,ISS,GTC,1.534553e+09,1079ca1e-c45d-424d-b742-e689f897335c,limit
5,BUY_ORDER,5b776add2d105a029aabbb49,2018-08-18T00:39:56.955529Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5739.10000000,BTC-EUR,False,buy,0.00100000,pending,dc,ISS,GTC,1.534553e+09,37ab041f-f091-4819-b820-a891c1bf09ea,limit


In [670]:
 db.delete_many( { "MONGOKEY" : "BUY_ORDER" } )

In [514]:
t = pd.to_datetime(time.time()-100,unit='s')
t

Timestamp('2018-08-17 20:35:59.502279999')

In [508]:
df['created_at'][3]

'2018-08-17T20:26:56.739788Z'

In [516]:
openorders = client.get_orders()[0]

In [622]:
df_openorders = pd.DataFrame(openorders)
df_openorders

""


In [621]:
df_openorders = pd.DataFrame(openorders).query('product_id == "BTC-EUR"')
df_openorders

UndefinedVariableError: name 'product_id' is not defined

In [520]:
df_openorders['created_at'] = pd.to_datetime(df_openorders['created_at'])

,created_at,executed_value,fill_fees,filled_size,id,post_only,price,product_id,settled,side,size,status,time_in_force,type
0,2018-08-17 20:37:29.299244,0.0000000000000000,0.0000000000000000,0.00000000,6451f256-a893-44ab-8e87-2004781a902e,False,4000.00000000,BTC-EUR,False,buy,0.00500000,open,GTC,limit


In [668]:
pd.DataFrame(list(db.find({'MONGOKEY':'FBP_UPDATE'})))

,MONGOKEY,_id,ds_fcst_0002,ds_fcst_002,product_id,timestamp,trend_fcst_0002,trend_fcst_002,yhat_fcst_0002,yhat_fcst_002,yhat_lower_fcst_0002,yhat_lower_fcst_002,yhat_upper_fcst_0002,yhat_upper_fcst_002
0,FBP_UPDATE,5b76240413ab1f0001d8940c,2018-08-17 01:26:00,2018-08-17 01:26:00,NaN,1.534469e+09,6306.997060,6356.773610,6306.997060,6356.773610,6132.051410,6338.034912,6469.843063,6375.703768
1,FBP_UPDATE,5b76242713ab1f0001d8940d,2018-08-17 01:26:00,2018-08-17 01:26:00,NaN,1.534469e+09,6306.997060,6356.773610,6306.997060,6356.773610,6142.399889,6337.900144,6472.756167,6375.028388
2,FBP_UPDATE,5b76244a13ab1f0001d8940e,2018-08-17 01:27:00,2018-08-17 01:27:00,NaN,1.534469e+09,6286.782316,6357.216501,6286.782316,6357.216501,6005.158032,6338.441925,6587.105787,6375.589092
3,FBP_UPDATE,5b762468400f86000155152e,2018-08-17 01:27:00,2018-08-17 01:27:00,NaN,1.534469e+09,6212.109638,6356.270813,6212.109638,6356.270813,5888.290439,6336.878098,6559.078596,6376.974452
4,FBP_UPDATE,5b76246b1aba1e00012e253e,2018-08-17 01:27:00,2018-08-17 01:27:00,NaN,1.534469e+09,6212.109638,6356.270813,6212.109638,6356.270813,5883.710736,6336.112153,6553.147420,6376.891130
5,FBP_UPDATE,5b76246dc86bc30001703d45,2018-08-17 01:27:00,2018-08-17 01:27:00,NaN,1.534469e+09,6212.109638,6356.270813,6212.109638,6356.270813,5881.034098,6334.219306,6545.501341,6375.668371
6,FBP_UPDATE,5b7624736fadab00015e3ea8,2018-08-17 01:27:00,2018-08-17 01:27:00,NaN,1.534469e+09,6212.109638,6356.270813,6212.109638,6356.270813,5874.055963,6335.936249,6555.038955,6375.535117
7,FBP_UPDATE,5b76247413ab1f0001d8940f,2018-08-17 01:27:00,2018-08-17 01:27:00,NaN,1.534469e+09,6320.622338,6357.430428,6320.622338,6357.430428,6091.363339,6339.517370,6550.693098,6375.177106
8,FBP_UPDATE,5b76249d13ab1f0001d89410,2018-08-17 01:28:00,2018-08-17 01:28:00,NaN,1.534469e+09,6269.574433,6356.844693,6269.574433,6356.844693,6016.534716,6334.946823,6530.594632,6380.151837
9,FBP_UPDATE,5b7624c513ab1f0001d89411,2018-08-17 01:29:00,2018-08-17 01:29:00,NaN,1.534469e+09,6259.072801,6359.249776,6259.072801,6359.249776,5946.091792,6342.227409,6543.052678,6376.579087


In [667]:

l = []
with db.watch() as stream:
        for change in stream:
            print(change)
            l.append(change)

{'_id': {'_data': '825B775CCD0000000129295A10041D485BD8E5F24A7F97FF3EFC75D0DBDB463C5F6964003C33343834373732642D363332332D346661372D386637612D326532663364363031626334000004'}, 'operationType': 'insert', 'clusterTime': Timestamp(1534549197, 1), 'fullDocument': {'_id': '3484772d-6323-4fa7-8f7a-2e2f3d601bc4', 'type': 'received', 'order_type': 'market', 'size': 0.012, 'side': 'sell', 'funds': 78.63576, 'product_id': 'BTC-USD', 'sequence': 6642208702, 'time': '2018-08-17T23:39:57.677000Z', 'y': 6552.9800000000005, 'timestamp': 1534549197.6827226, 'MONGOKEY': 'MARKET_UPDATE'}, 'ns': {'db': 'gdax', 'coll': 'gdaxws'}, 'documentKey': {'_id': '3484772d-6323-4fa7-8f7a-2e2f3d601bc4'}}
{'_id': {'_data': '825B775CCE0000000129295A10041D485BD8E5F24A7F97FF3EFC75D0DBDB463C5F6964003C32623866353862392D353335362D343836652D623864612D666666643131313038633266000004'}, 'operationType': 'insert', 'clusterTime': Timestamp(1534549198, 1), 'fullDocument': {'_id': '2b8f58b9-5356-486e-b8da-fffd11108c2f', 'type': 'rec

KeyboardInterrupt: 